In [107]:
# Import the library
from hgboost import hgboost

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from skopt import BayesSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [130]:
# Preparing the data
df = pd.read_csv('df_augmented.csv')
df.drop(['Unnamed: 0', 'taus_out', 'taua_out'], axis=1, inplace=True)

# reading test data
df_test = pd.read_csv('df_test.csv')
df_test.drop('Unnamed: 0', axis=1, inplace=True)

df

,N,xm,nr,ni,p1,p2,p3
0,7,0.640,1.71,0.045,1.685110,-0.001577,-0.009044
1,10,0.336,1.30,0.023,2.235463,-0.096444,-0.104801
2,6,0.640,1.30,0.023,0.339716,-0.000123,-0.002486
3,7,0.480,1.30,0.023,0.618308,-0.000189,-0.001309
4,8,0.530,1.80,0.023,1.429310,-0.007968,-0.013564
...,...,...,...,...,...,...,...
388,9,0.400,1.90,0.066,10.858241,-0.306511,-0.570028
389,9,0.600,1.88,0.077,8.596835,-0.039968,-0.198187
390,7,0.500,1.79,0.029,4.598836,-0.169376,0.012427
391,9,0.400,2.02,0.022,1.703093,-0.055314,-0.186390


In [109]:
# from bhmie import *

# data = pd.read_csv('allregimes.csv')

# def calc_tau(N, xm, nr, ni):
#     N = 2**N
#     s1,s2,qext,qsca,qback,gsca = bhmie(xm,complex(nr, ni),91)
#     csca_mon = qsca * np.pi * xm**2
#     cext_mon = qext * np.pi * xm**2
#     cabs_mon = (qext - qsca) * np.pi * xm**2

#     R1 = 1.598
#     Rcut = 1. / 0.0001
#     Df = 2.0
#     Rmin=2.0
#     #dstep = 1/(8*xm)
#     ds = 1/(8*xm)
#     Rmax = R1*(N*np.log(Rcut))**(1.0/Df)
#     nstep = int((Rmax - Rmin) /ds)
#     nstep = np.where(nstep < 100, 100, nstep)
#     dstep = (Rmax - Rmin)/(nstep-1)

#     R0 = np.arange(nstep)*dstep + Rmin
#     dist = R0 * xm
#     nc = ((1.0 - np.exp(-(R0/R1)**(Df) /N))*(1.0 - np.exp(-(R0/3.478)**3.194)) + (2.0/N) )/(1 + (2.0/N))
#     #nc=1.0 - np.exp(-(R0/R1)**(Df) /N)
#     f0 = np.zeros(nstep)
#     f0[0] = 0.5 * (nc[1]-nc[0]) + nc[0] # this seems to include nc[0] erronously
#     #f0[0] = 0.5 * (nc[1]-nc[0])
#     f0[nstep-1] =  1. - nc[nstep-1] + 0.5 * (nc[nstep-1]-nc[nstep-2])
#     #f0[nstep-1] = 0.5 * (nc[nstep-1]-nc[nstep-2]) #
#     f0[1:(nstep-1)] =  (nc[2:(nstep)] - nc[0:(nstep-2)])/(2)
#     tau_coef = np.sum(f0 / dist**2) / 4. / np.pi * (N-1)
#     taue_out = tau_coef * cext_mon
#     taus_out = tau_coef * csca_mon
#     taua_out = tau_coef * cabs_mon
#     return taus_out, taua_out


In [114]:
# tau_new = df_test.apply(lambda row: pd.Series(calc_tau(row['N'], row['xm'], row['nr'], row['ni'])), axis=1).values
# taus_out = tau_new[:,0]
# taua_out = tau_new[:,1]
# df_test[['taus_out', 'taua_out']] = tau_new

#### Modelling Strategy
For each observation there are three response variables. The response variables, p1, p2, p3 are ceofficients of a legendre polynomial describing the same curve of a phase function that corresponds to an aerosol particle with features N, xm, nr, ni. Therefore each response variable depend on the features as well as each other. Finally the relationship between the the features and response variable may not be linear.

Based on these properties of the data using a tree based algorithm I will first model p1 using the features in the data 

# Step 1 Modelling P1

In [132]:
# For p1 do not use any p columns as predictor 
X = df.drop(columns=['p1', 'p2', 'p3'])
y1 = df['p1'].values

X_test = df_test.drop(columns=['p1', 'p2', 'p3'])
y_test1 = df_test['p1']

In [133]:
# Define the search space for hyperparameters
params = {
    'learning_rate': (0.0005, 1.0, 'log-uniform'),
    'n_estimators': (10, 5000),
    'max_depth': (1, 50),
    'min_child_weight': (1, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'colsample_bytree': (0.1, 1.0, 'uniform')
}

# Define the XGBoost regression model
xgb = XGBRegressor()

In [134]:
# Define the BayesSearchCV object for hyperparameter tuning
bayes_cv1 = BayesSearchCV(
    estimator=xgb,
    search_spaces=params,
    cv=4,
    n_iter=350,
    n_jobs=-1,
    verbose=0
)

# Fit the BayesSearchCV object to the training data
bayes_cv1.fit(X, y1)


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scip

BayesSearchCV(cv=4,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                        

In [135]:
# Print the best set of hyperparameters found by the optimization algorithm
print("Best hyperparameters: ", bayes_cv1.best_params_)


Best hyperparameters:  OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.00155552111236498), ('max_depth', 50), ('min_child_weight', 1), ('n_estimators', 3698), ('subsample', 0.2991309429583223)])


In [136]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate the performance of the best model on the test set
y_pred = bayes_cv1.predict(X_test)
mae = mean_absolute_error(y_test1, y_pred)
rmse = mean_squared_error(y_test1, y_pred, squared=False)
r2 = r2_score(y_test1, y_pred)

print("Mean Absolute Error: {:.3f}".format(mae))
print("Root Mean Squared Error: {:.3f}".format(rmse))
print("R-squared Score: {:.3f}".format(r2))

Mean Absolute Error: 0.580
Root Mean Squared Error: 0.972
R-squared Score: 0.948


In [137]:
# Save the best model to disk
import joblib
joblib.dump(bayes_cv1.best_estimator_, 'xgb_model_p1.joblib')

['xgb_model_p1.joblib']

In [138]:
y_residual1.shape

(393,)

## Second model - P2
For model P2 we use the standard features and p1 as predictor

In [139]:
# 
X2 = df.drop(columns=['p2', 'p3'])
X_test2 = df_test.drop(columns=['p2', 'p3'])

y2 = df['p2'].values
y_test2 = df_test['p2']

In [140]:
# Define the BayesSearchCV object for hyperparameter tuning
bayes_cv2 = BayesSearchCV(
    estimator=xgb,
    search_spaces=params,
    cv=3,
    n_iter=250,
    n_jobs=-1,
    verbose=1
)

# Fit the BayesSearchCV object to the training data
bayes_cv2.fit(X2, y2 )

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

BayesSearchCV(cv=3,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                        

In [141]:
# Print the best set of hyperparameters found by the optimization algorithm
print("Best hyperparameters: ", bayes_cv2.best_params_)


Best hyperparameters:  OrderedDict([('colsample_bytree', 0.6793130680658069), ('learning_rate', 0.0031483740130653108), ('max_depth', 29), ('min_child_weight', 4), ('n_estimators', 2832), ('subsample', 0.4056080431404968)])


In [142]:
joblib.dump(bayes_cv1.best_estimator_, 'xgb_model_p2.joblib')

['xgb_model_p2.joblib']

In [143]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate the performance of the best model on the test set
y_pred2 = bayes_cv2.predict(X_test2)
mae = mean_absolute_error(y_test2, y_pred2)
rmse = mean_squared_error(y_test2, y_pred2, squared=False)
r2 = r2_score(y_test2, y_pred2)

print("Mean Absolute Error: {:.3f}".format(mae))
print("Root Mean Squared Error: {:.3f}".format(rmse))
print("R-squared Score: {:.3f}".format(r2))

Mean Absolute Error: 0.094
Root Mean Squared Error: 0.226
R-squared Score: 0.815


# Third Model - P3
For P3 we use standard features plus P1 and P2 as predictors

In [144]:
# Dropping only P3 leaves p1 and p2 among the predictors
X3 = df.drop(columns=['p3'])
X_test3 = df_test.drop(columns=['p3'])

y3 = df['p3'].values
y_test3 = df_test['p3']

# Define the BayesSearchCV object for hyperparameter tuning
bayes_cv3 = BayesSearchCV(
    estimator=xgb,
    search_spaces=params,
    cv=3,
    n_iter=250,
    n_jobs=-1,
    verbose=0
)

# Fit the BayesSearchCV object to the training data
bayes_cv3.fit(X3, y3 )

BayesSearchCV(cv=3,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                        

In [145]:
# Print the best set of hyperparameters found by the optimization algorithm
print("Best hyperparameters: ", bayes_cv3.best_params_)


Best hyperparameters:  OrderedDict([('colsample_bytree', 0.8076593590960413), ('learning_rate', 0.003276365902103458), ('max_depth', 49), ('min_child_weight', 1), ('n_estimators', 5000), ('subsample', 0.19704264967965748)])


In [146]:
joblib.dump(bayes_cv1.best_estimator_, 'xgb_model_p3.joblib')

['xgb_model_p3.joblib']

In [147]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate the performance of the best model on the test set
y_pred3 = bayes_cv3.predict(X_test3)
mae = mean_absolute_error(y_test3, y_pred3)
rmse = mean_squared_error(y_test3, y_pred3, squared=False)
r2 = r2_score(y_test3, y_pred3)

print("Mean Absolute Error: {:.3f}".format(mae))
print("Root Mean Squared Error: {:.3f}".format(rmse))
print("R-squared Score: {:.3f}".format(r2))

Mean Absolute Error: 0.017
Root Mean Squared Error: 0.031
R-squared Score: 0.970
